In [1]:
%matplotlib inline

Source: [https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words)

# Word Embeddings: Encoding Lexical Semantics

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


# Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


# Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


# An Example: N-Gram Language Modeling

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.




In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[519.7060680389404, 517.0581159591675, 514.4285066127777, 511.81494069099426, 509.21706771850586, 506.63392329216003, 504.063987493515, 501.50686407089233, 498.9624342918396, 496.4288454055786]


# Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




In [5]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):

    def __init__(self,vocab_size,embedding_dim, context_size):
        super(CBOW,self).__init__()
        #set embbedings
        self.embeddings=nn.Embedding(vocab_size, embedding_dim)
        #set linear
        self.linear = nn.Linear(context_size * embedding_dim, vocab_size)

    def forward(self, inputs):
        batch_size = inputs.shape[0]
        embedded_context  = self.embeddings(inputs)
        embedded_context  = embedded_context.view(batch_size,-1)
        output = self.linear(embedded_context)
        log_probs = F.log_softmax(output,dim=-1)
        return log_probs

EMBEDDING_DIM=10
losses = []
# Define loss function (e.g., negative log likelihood)
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE * 2)
# Define optimizer (e.g., stochastic gradient descent)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# create your model and train.  here are some functions to help you make
# the data ready for use by your module

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)
# make_context_vector(data[0][0], word_to_ix)  # example

batch_size = 1
batches = [data[i:i + batch_size] for i in range(0, len(data), batch_size)]
epoch_runs = 10
for epoch in range(epoch_runs):
  total_loss = 0
  for batch in batches:
    context = torch.stack([make_context_vector(b[0], word_to_ix) for b in batch], dim=0)
    target = torch.cat([make_context_vector([b[1]],word_to_ix) for b in batch], dim=0)

    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass
    log_prob = model(context)
    # Calculate the loss for each context
    loss = loss_function(log_prob, target)
    # Backward pass (compute gradients)
    loss.backward()
    # Update the model's parameters using the optimizer
    optimizer.step()
    total_loss += loss.item()
  print(f'Epoch [{epoch + 1}/{epoch_runs}], Loss: {total_loss}')

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
Epoch [1/10], Loss: 232.43795371055603
Epoch [2/10], Loss: 209.57329416275024
Epoch [3/10], Loss: 187.89076590538025
Epoch [4/10], Loss: 167.48481786251068
Epoch [5/10], Loss: 148.47695398330688
Epoch [6/10], Loss: 130.9853950738907
Epoch [7/10], Loss: 115.0912761092186
Epoch [8/10], Loss: 100.82341784238815
Epoch [9/10], Loss: 88.16429269313812
Epoch [10/10], Loss: 77.05667462944984


# Part 1: Train CBOW embeddings for both datasets

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
%cd /content/gdrive/My\ Drive/NLP/ex2
%pwd

/content/gdrive/My Drive/NLP/ex2


'/content/gdrive/My Drive/NLP/ex2'

## CBOW2 with a context width of 2 (in both directions) for the Hotel Reviews dataset

In [8]:
import pandas as pd
import string
import re

In [9]:
# punctuation_mark = string.punctuation
# punctuation_mark_replacement = " " * len(punctuation_mark)

def preprocess(text):
  text = str(text).lower()
  # text = text.translate(str.maketrans(punctuation_mark, punctuation_mark_replacement))
  text = re.sub(r'[^a-z]', ' ', text)
  return text


In [10]:
def convertWordIntoCBOW(text, cbow_size):
  text = text.split()
  data = []
  for i in range(cbow_size, len(text) - cbow_size):
      _previous = [text[i-(cbow_size-j)] for j in range(cbow_size)]
      _after = [text[i+(j+1)] for j in range(cbow_size)]
      context = _previous + _after
      target = text[i]
      data.append((context, target))
  return data

In [11]:
df = pd.read_csv("tripadvisor_hotel_reviews.csv")

In [12]:
df["Preprocess_Review"] = df["Review"].apply(lambda x: preprocess(x))

In [13]:
raw_text = " ".join(df["Preprocess_Review"].to_list()).split()
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}

In [14]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
df["cbow2"] = df["Preprocess_Review"].apply(lambda x: convertWordIntoCBOW(str(x), CONTEXT_SIZE))

In [15]:
df["cbow2_context_vector"] = df["cbow2"].apply(lambda x: torch.stack([make_context_vector(_c[0], word_to_ix) for _c in x], dim=0))

In [16]:
df["cbow2_target_vector"] = df["cbow2"].apply(lambda x: torch.cat([make_context_vector([_c[1]], word_to_ix) for _c in x], dim=0))

In [17]:
all_context = torch.vstack(df["cbow2_context_vector"].tolist()) # Shape: [2081043, 4]
all_target = torch.cat(df["cbow2_target_vector"].tolist()) # Shape: [2081043]

In [18]:
import numpy as np

EMBEDDING_DIM= 50
losses = []
# Define loss function (e.g., negative log likelihood)
loss_function = nn.NLLLoss()
model1 = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE * 2)
# Define optimizer (e.g., stochastic gradient descent)
optimizer = optim.SGD(model1.parameters(), lr=0.1)
epoch_runs = 15

for epoch in range(epoch_runs):
  # Select random number for mini-batch
  indices = np.random.randint(0, len(all_context), size=100000)
  mini_context = torch.index_select(all_context, 0, torch.tensor(indices))
  mini_target = torch.index_select(all_target, 0, torch.tensor(indices))

  batch_size = 10000
  batches_context = [mini_context[i:i + batch_size] for i in range(0, len(mini_context), batch_size)]
  batches_target = [mini_target[i:i + batch_size] for i in range(0, len(mini_target), batch_size)]

  total_loss = 0
  for i in range(len(batches_context)):
    _context = batches_context[i]
    _target = batches_target[i]
    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass
    log_prob = model1(_context)
    # Calculate the loss for each context
    loss = loss_function(log_prob, _target)
    # Backward pass (compute gradients)
    loss.backward()
    # Update the model's parameters using the optimizer
    optimizer.step()
    total_loss += loss.item()
  print(f'Epoch [{epoch + 1}/{epoch_runs}], Loss: {total_loss}')

Epoch [1/15], Loss: 109.61113357543945
Epoch [2/15], Loss: 109.4278211593628
Epoch [3/15], Loss: 109.31312942504883
Epoch [4/15], Loss: 109.14944076538086
Epoch [5/15], Loss: 109.02860832214355
Epoch [6/15], Loss: 108.89346218109131
Epoch [7/15], Loss: 108.75112438201904
Epoch [8/15], Loss: 108.57921981811523
Epoch [9/15], Loss: 108.42204856872559
Epoch [10/15], Loss: 108.27514171600342
Epoch [11/15], Loss: 108.12121105194092
Epoch [12/15], Loss: 108.01664447784424
Epoch [13/15], Loss: 107.85929679870605
Epoch [14/15], Loss: 107.71854496002197
Epoch [15/15], Loss: 107.51247882843018


## CBOW5 with a context width of 5 (in both directions) for the Hotel Reviews dataset

In [19]:
df2 = df # Dataframe2 is for CBOW5

In [20]:
# Split the "Preprocess_Review" column into words and count the number of words in each row
df2['word_count'] = df2['Preprocess_Review'].str.split().str.len()
# Filter the rows where the word count is more than 10
df2 = df2[df2['word_count'] > 10]

In [ ]:
CONTEXT_SIZE = 5  # 5 words to the left, 5 to the right
df2["cbow5"] = df2["Preprocess_Review"].apply(lambda x: convertWordIntoCBOW(str(x), CONTEXT_SIZE))

In [ ]:
df2["cbow5_context_vector"] = df2["cbow5"].apply(lambda x: torch.stack([make_context_vector(_c[0], word_to_ix) for _c in x], dim=0))
df2["cbow5_target_vector"] = df2["cbow5"].apply(lambda x: torch.cat([make_context_vector([_c[1]], word_to_ix) for _c in x], dim=0))

In [23]:
all_context = torch.vstack(df2["cbow5_context_vector"].tolist())
all_target = torch.cat(df2["cbow5_target_vector"].tolist())

In [24]:
import numpy as np

EMBEDDING_DIM= 50
losses = []
# Define loss function (e.g., negative log likelihood)
loss_function = nn.NLLLoss()
model2 = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE * 2)
# Define optimizer (e.g., stochastic gradient descent)
optimizer = optim.SGD(model2.parameters(), lr=0.1)
epoch_runs = 15

# print(f'Split into how many batches: {len(batches_context)}')

for epoch in range(epoch_runs):
  # Select random number for mini-batch
  indices = np.random.randint(0, len(all_context), size=100000)
  mini_context = torch.index_select(all_context, 0, torch.tensor(indices))
  mini_target = torch.index_select(all_target, 0, torch.tensor(indices))

  batch_size = 10000
  batches_context = [mini_context[i:i + batch_size] for i in range(0, len(mini_context), batch_size)]
  batches_target = [mini_target[i:i + batch_size] for i in range(0, len(mini_target), batch_size)]

  total_loss = 0
  for i in range(len(batches_context)):
    _context = batches_context[i]
    _target = batches_target[i]
    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass
    log_prob = model2(_context)
    # Calculate the loss for each context
    loss = loss_function(log_prob, _target)
    # Backward pass (compute gradients)
    loss.backward()
    # Update the model's parameters using the optimizer
    optimizer.step()
    total_loss += loss.item()
  print(f'Epoch [{epoch + 1}/{epoch_runs}], Loss: {total_loss}')

Epoch [1/15], Loss: 109.61921787261963
Epoch [2/15], Loss: 109.47204399108887
Epoch [3/15], Loss: 109.34921932220459
Epoch [4/15], Loss: 109.14020156860352
Epoch [5/15], Loss: 109.01509475708008
Epoch [6/15], Loss: 108.8316421508789
Epoch [7/15], Loss: 108.664231300354
Epoch [8/15], Loss: 108.56079864501953
Epoch [9/15], Loss: 108.42676830291748
Epoch [10/15], Loss: 108.19740390777588
Epoch [11/15], Loss: 108.06957340240479
Epoch [12/15], Loss: 107.91314125061035
Epoch [13/15], Loss: 107.7746171951294
Epoch [14/15], Loss: 107.60156536102295
Epoch [15/15], Loss: 107.49995422363281


## CBOW2 with a context width of 2 (in both directions) for the Sci-Fi story dataset

In [25]:
with open("scifi.txt", "r", encoding="utf-8") as file:
    _file = file.read()
    tokens_sci = preprocess(_file).split()

In [26]:
tokens_sci = tokens_sci[:500000]
raw_text_sci = " ".join(tokens_sci)

In [27]:
vocab_sci = set(tokens_sci)
vocab_size = len(vocab_sci)
word_to_ix_sci = {word: i for i, word in enumerate(vocab_sci)}

In [28]:
vocab_size

24018

In [29]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
data_sci = convertWordIntoCBOW(raw_text_sci, CONTEXT_SIZE)

In [30]:
EMBEDDING_DIM=50
losses = []
# Define loss function (e.g., negative log likelihood)
loss_function = nn.NLLLoss()
model3 = CBOW(len(vocab_sci), EMBEDDING_DIM, CONTEXT_SIZE * 2)
# Define optimizer (e.g., stochastic gradient descent)
optimizer = optim.SGD(model3.parameters(), lr=0.1)

batch_size = 5000
batches = [data_sci[i:i + batch_size] for i in range(0, len(data_sci), batch_size)]
epoch_runs = 3
for epoch in range(epoch_runs):
  total_loss = 0
  for batch in batches:
    context = torch.stack([make_context_vector(b[0], word_to_ix_sci) for b in batch], dim=0)
    target = torch.cat([make_context_vector([b[1]],word_to_ix_sci) for b in batch], dim=0)

    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass
    log_prob = model3(context)
    # Calculate the loss for each context
    loss = loss_function(log_prob, target)
    # Backward pass (compute gradients)
    loss.backward()
    # Update the model's parameters using the optimizer
    optimizer.step()
    total_loss += loss.item()
  print(f'Epoch [{epoch + 1}/{epoch_runs}], Loss: {total_loss}')

Epoch [1/3], Loss: 1007.466796875
Epoch [2/3], Loss: 971.5434722900391
Epoch [3/3], Loss: 942.5203304290771


# Part 2: Test Embeddings

## Task 1


In [31]:
import torch.nn as nn
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
# Define the function for nearest neighbour computation
def get_closest_word(word, net, vocabulary, topn=5):
    word_distance = []
    emb = net.embeddings
    pdist = nn.PairwiseDistance()

    word_to_ix = {word: i for i, word in enumerate(vocabulary)}
    ix_to_word = {i: word for word, i in word_to_ix.items()}

    i = word_to_ix[word]
    lookup_tensor_i = torch.tensor([i], dtype=torch.long)
    v_i = emb(lookup_tensor_i)

    for j in range(len(vocabulary)):
        if j != i:
            lookup_tensor_j = torch.tensor([j], dtype=torch.long)
            v_j = emb(lookup_tensor_j)
            # index_to_word = {i: word for word, i in word_to_index.items()}
            word_distance.append((ix_to_word[j], float(pdist(v_i, v_j))))

    word_distance.sort(key=lambda x: x[1])
    return word_distance[:topn]

For the hotel reviews dataset, the words we choose are the following:  
3 nouns: hotel, room, location;    
3 verbs: recommend, eat, swim;    
3 adjectives: nice, comfortable, terrible.     
Then we train with the CBOW2 model.

In [33]:
# List of 9 chosen words
word_hotel = ["hotel", "room", "location", "recommend", "eat", "swim", "nice", "comfortable", "terrible"]

# model1 = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE * 2)
print("For the Hotel Review Dataset: ")
for word in word_hotel:
    closest_words = get_closest_word(word, model1, vocab, topn=5)
    print(f"5 Closest words to '{word}': {closest_words}")

For the Hotel Review Dataset: 
5 Closest words to 'hotel': [('lion', 6.514893054962158), ('putting', 6.883223533630371), ('security', 6.994818687438965), ('honda', 6.997636318206787), ('confimed', 7.024110317230225)]
5 Closest words to 'room': [('treatet', 6.392022609710693), ('ripiao', 6.436098098754883), ('relativeley', 6.463700294494629), ('unluckyas', 6.490079879760742), ('devils', 6.5072855949401855)]
5 Closest words to 'location': [('jazzuzi', 6.675212860107422), ('jars', 6.971132278442383), ('cuppas', 7.025022506713867), ('europan', 7.169882297515869), ('forthcoming', 7.177751064300537)]
5 Closest words to 'recommend': [('mussed', 6.742995262145996), ('bohia', 6.812859058380127), ('voorburgwal', 6.839596748352051), ('feliz', 6.946531772613525), ('goodpueblo', 7.018956184387207)]
5 Closest words to 'eat': [('unfavourable', 6.214866638183594), ('occidental', 6.54563570022583), ('dicky', 6.55186128616333), ('serinis', 6.668513298034668), ('ayung', 6.717133522033691)]
5 Closest word

## Task 2
For the Sci-fi dataset, the words we choose are the following:  
3 nouns: life, boots, costume;  
3 verbs: speak, tell, lose;  
3 adjectives: interesting, strange, satisfied.  
Then we also train with the CBOW2 model.


In [34]:
# List of 9 chosen words
word_sci = ["life", "boots", "costume", "speak", "tell", "lose", "interesting", "strange", "satisfied"]

print("For the Sci-fi Dataset: ")
for word in word_sci:
    closest_words = get_closest_word(word, model3, vocab_sci, topn=5)
    print(f"5 Closest words to '{word}': {closest_words}")

For the Sci-fi Dataset: 
5 Closest words to 'life': [('pessimistic', 7.025411605834961), ('prodigious', 7.352947235107422), ('account', 7.521966934204102), ('accusations', 7.5261125564575195), ('hotly', 7.74095344543457)]
5 Closest words to 'boots': [('teacher', 7.195479869842529), ('whoie', 7.213236331939697), ('excellently', 7.436926364898682), ('generator', 7.444934844970703), ('angled', 7.497432708740234)]
5 Closest words to 'costume': [('eveiything', 6.489418983459473), ('oncfe', 6.501927852630615), ('whoie', 6.568801403045654), ('shocked', 6.628296375274658), ('gurruls', 6.698210716247559)]
5 Closest words to 'speak': [('repellent', 6.5739336013793945), ('lamentable', 6.8800129890441895), ('medic', 7.001706123352051), ('negro', 7.006081581115723), ('sand', 7.021505355834961)]
5 Closest words to 'tell': [('donnelfs', 6.734241008758545), ('poker', 6.938412189483643), ('height', 6.977206230163574), ('always', 6.97877836227417), ('pondwater', 7.015625)]
5 Closest words to 'lose': [('

## Task 5
We select a word that appears frequently and a word that appears very rarely.  
The words we choose are: furnished and bridge  
First, we check that these two words are present in both data sets, and then we find their five closest neighbors in both data sets.  


In [35]:
selected_words = ["furnished", "bridge"]

for word in selected_words:
    if word not in vocab:
      print(f"'{word}' does not exist in the hotel review dataset.")
    if word not in vocab_sci:
      print(f"'{word}' does not exist in the sci-fi dataset.")
    else:
      print(f"'{word}' exists in both datasets.")

'furnished' exists in both datasets.
'bridge' exists in both datasets.


In [36]:
for word in selected_words:
    closest_words_hotel = get_closest_word(word, model1, vocab, topn=5)
    closest_words_sci = get_closest_word(word, model3, vocab_sci, topn=5)
    print(f"5 Closest words to '{word}' in the hotel review dataset: {closest_words_hotel}")
    print(f"5 Closest words to '{word}' in the sci-fi dataset: {closest_words_sci}")

5 Closest words to 'furnished' in the hotel review dataset: [('sobeys', 7.069890975952148), ('smok', 7.0742316246032715), ('veryconvenient', 7.097971439361572), ('peppito', 7.153048038482666), ('adversities', 7.291590213775635)]
5 Closest words to 'furnished' in the sci-fi dataset: [('buffet', 6.332549095153809), ('defenders', 6.49399995803833), ('extermination', 6.504188060760498), ('jez', 6.509891986846924), ('mother', 6.556239604949951)]
5 Closest words to 'bridge' in the hotel review dataset: [('backpacked', 6.839584827423096), ('brandenberg', 6.98991060256958), ('kayakingwe', 7.158514022827148), ('jeffhanman', 7.168525218963623), ('descartes', 7.18250846862793)]
5 Closest words to 'bridge' in the sci-fi dataset: [('gunpowder', 5.334932804107666), ('root', 6.524269104003906), ('genarion', 6.703142166137695), ('klia', 6.730693340301514), ('telecontrolled', 6.932901382446289)]
